In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from scipy.stats import spearmanr
from scipy.stats import norm

import statsmodels.api as sm
from datetime import datetime
from bisect import bisect_left, bisect_right

Meteorological and Hydrological time series and Annual maxima sets

In [ ]:
conf = pd.read_csv(r"C:\Users\Fonse_ma\OneDrive - Stichting Deltares\Desktop\Maria\Notebooks\HBV\Confluences_ok2.csv", delimiter=';')

Meteorological

In [ ]:
#File j    25       50000/25 = 2000 years /file
j = 5
path1 = r"F:\MET\TimeSeries\ds%d" %j
#Ensemble   20       2000/20 = 100 years/ensemble
i = 0
path2 = r"_%draina.npy" %i
path = path1 + path2
ds_rain = np.load(path, allow_pickle=True).item()
# ds_rain

In [ ]:
MS = 'C67'
TS = 'C66'
w = 5
df_rain = pd.DataFrame(columns=['P_S1', 'P_S2'])
df_rain['P_S1'] = ds_rain[MS]  #mm/day
df_rain['P_S2'] = ds_rain[TS]  #mm/day
df_rain['P_S1 m3/sec'] = df_rain['P_S1'] * 0.000001 * 1000000000 / 864000
df_rain['P_S2 m3/sec'] = df_rain['P_S2'] * 0.000001 * 1000000000 / 864000
df_rain

In [ ]:
start = 500 
end = 1000
plt.figure(figsize=(20, 5))
plt.plot(df_rain['P_S1 m3/sec'][start:end], 'g', label='1 day')
# plt.plot(df_rain['P_S1'][start:end], 'g', label='1 day')
# plt.plot(df_rain['P_S1rol m3/sec'][start:end], 'g', label='5 days')

plt.title(f'Mainstream (MS) \nPrecipitation', fontsize=22) #Runoff
plt.xlabel('Day', fontsize=14)
plt.ylabel('Precipitation (m3/sec)', fontsize=14) # Q m3/s
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(0, 3600)
# plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid()

In [ ]:
start = 500 
end = 1000
plt.figure(figsize=(20, 5))
# plt.plot(df_rain['P_S2'][start:end], 'g', label='1 day')
plt.plot(df_rain['P_S2 m3/sec'][start:end], 'g', label='1 day')
# plt.plot(df_rain['P_S1rol m3/sec'][start:end], 'g', label='5 days')

plt.title(f'Tributary stream (TS) \nPrecipitation', fontsize=22) #Runoff
plt.xlabel('Day', fontsize=14)
plt.ylabel('Precipitation (m3/sec)', fontsize=14) # Q m3/s
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylim(0, 3600)
# plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid()

In [ ]:
desc = 'rain'

# op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AMmet\C%d_" %Cnum[c] + desc + r"_" 
op = r"P:\11206883-006-dar-cloud-computing\RhineFiles\MET\AMmet\C67_" + desc + r"_" 
AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

In [ ]:

unit = 'm3/sec'
desc = 'disc1d'
c = 67
peaks = 50
color = 'g'

AM1['ind'] = np.arange(len(AM1))
AM2['ind'] = np.arange(len(AM2))
AM3['ind'] = np.arange(len(AM3))
rho1, Sp1 = spearmanr(AM1['P_S1rol'], AM1['P_S2rol'])
rho2, Sp2 = spearmanr(AM2['P_S1rol'], AM2['P_S2rol'])
rho3, Sp3 = spearmanr(AM3['P_S1rol'], AM3['P_S2rol'])

randomindex1 = np.random.choice(AM1.ind, size=peaks, replace=False)
set1 = pd.DataFrame()
set2 = pd.DataFrame()
set3 = pd.DataFrame()
for i in range(peaks):
    set1 = set1.append(AM1.loc[AM1.ind == randomindex1[i]])
    set2 = set2.append(AM2.loc[AM2.ind == randomindex1[i]])
    set3 = set3.append(AM3.loc[AM3.ind == randomindex1[i]])

a =  0.000001 * 1000000000 / 864000
plt.figure(figsize=(7, 7))
plt.plot(set1['P_S1'] * a , set1['P_S2']* a , 'o', color='none', markeredgecolor=color,markersize=10, label=f'Set 1: S1max-S2conc \n rho={rho1:.4f}')
# plt.plot(set2['P_S1'], set2['P_S2'], 'x', color=color, markersize=9, label=f'Set 2: S2max-S1conc \n rho={rho2:.4f}')
# plt.plot(set3['P_S1'], set3['P_S2'], '.', color=color, markersize=10, label=f'Set 3: Cmax (C=S1+S2) \n rho={rho3:.4f}')
plt.title(f'Precipitation \nAnnual Maxima Peaks', fontsize=20, y=1.05)
plt.xlabel(f'Mainstream (MS) \n (m3/sec)', fontsize=18)
plt.ylabel(f'Tributary stream (TS) \n (m3/sec)', fontsize=18)
# plt.xlim(left=0, right=16000)
# plt.ylim(ymin = 0, ymax =4000)
plt.xticks(fontsize=12, rotation=90)
plt.yticks(fontsize=12)
# plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")

plt.grid()

Hydrological

In [ ]:
c = 'C68'
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\01_TSonefile\TS_" + c + ".csv" 
TS_C68 = pd.read_csv(op, delimiter=';')

TS_C68 = TS_C68[~(TS_C68['Q_S1'] < 0)]
TS_C68 = TS_C68[~(TS_C68['Q_S2'] < 0)]
TS_C68 = TS_C68[~(TS_C68['Q_S2'] == 999)]
# TS_C68   

In [ ]:
TS_C68

In [ ]:
start=500
end=1000

columns = ['Q_S1']
data = TS_C68.loc[start:end, columns]

plt.figure(figsize=(20, 5))
plt.plot(TS_C68.Q_S1.loc[start:end], 'b', label='1 day')

plt.title(f'Mainstream (MS) \nDischarge', fontsize=22) #Runoff
plt.ylim(0, 4000)
plt.xlabel('Day', fontsize=14)
plt.ylabel('Discharge (m3/sec)', fontsize=14) # Q m3/s
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid()

In [ ]:
start=500
end=1000

columns = ['Q_S2']
data = TS_C68.loc[start:end, columns]

plt.figure(figsize=(20, 5))
plt.plot(TS_C68.Q_S2.loc[start:end], 'b', label='1 day')

plt.title(f'Tributary stream (TS)\nDischarge', fontsize=22) #Runoff
plt.xlabel('Day', fontsize=14)
plt.ylabel('Discharge (m3/sec)', fontsize=14) # Q m3/s
plt.ylim(0, 4000)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
# plt.legend(fontsize=12, bbox_to_anchor=(1,1), loc="upper left")
plt.grid()

In [ ]:
c = 68
op = r"P:\11206883-006-dar-cloud-computing\GRADE_export\HIDnew\05_HYDzoom" + r"\C%d_" %c + r"disc1d_"
    
AM1 = pd.read_pickle(op + "AM1.pkl")
AM2 = pd.read_pickle(op + "AM2.pkl")
AM3 = pd.read_pickle(op + "AM3.pkl")

In [ ]:
td = 'HYD | Discharge 1d'
unit = 'm3/sec'
desc = 'disc1d'
c = 67
peaks = 50
color = 'b'

AM1['ind'] = np.arange(len(AM1))
AM2['ind'] = np.arange(len(AM2))
AM3['ind'] = np.arange(len(AM3))
rho1, Sp1 = spearmanr(AM1['Q_S1'], AM1['Q_S2'])
rho2, Sp2 = spearmanr(AM2['Q_S1'], AM2['Q_S2'])
rho3, Sp3 = spearmanr(AM3['Q_S1'], AM3['Q_S2'])

randomindex1 = np.random.choice(AM1.ind, size=peaks, replace=False)
set1 = pd.DataFrame()
set2 = pd.DataFrame()
set3 = pd.DataFrame()
for i in range(peaks):
    set1 = set1.append(AM1.loc[AM1.ind == randomindex1[i]])
    set2 = set2.append(AM2.loc[AM2.ind == randomindex1[i]])
    set3 = set3.append(AM3.loc[AM3.ind == randomindex1[i]])


plt.figure(figsize=(7, 7))
plt.plot(set1['Q_S1'], set1['Q_S2'], 'o', color='none', markeredgecolor=color,markersize=10, label=f'Set 1: S1max-S2conc \n rho={rho1:.4f}')
# plt.plot(set2['Q_S1'], set2['Q_S2'], 'x', color=color, markersize=9, label=f'Set 2: S2max-S1conc \n rho={rho2:.4f}')
# plt.plot(set3['Q_S1'], set3['Q_S2'], '.', color=color, markersize=10, label=f'Set 3: Cmax (C=S1+S2) \n rho={rho3:.4f}')
plt.title(f'Discharge \nAnnual Maxima Peaks', fontsize=20, y=1.05)
plt.xlabel(f'Mainstream (MS) \n (m3/sec)', fontsize=18)
plt.ylabel(f'Tributary stream (TS) \n (m3/sec)', fontsize=18)
# plt.xlim(left=0, right=16000)
# plt.ylim(ymin = 0, ymax =4000)
plt.xticks(fontsize=12, rotation=90)
plt.yticks(fontsize=12)
# plt.legend(fontsize=18, bbox_to_anchor=(1,1), loc="upper left")

plt.grid()